In [7]:
import gensim
import numpy as np
import pandas as pd

from gensim.models.doc2vec import Doc2Vec, LabeledSentence

TaggededDocument = gensim.models.doc2vec.TaggedDocument

def test( test):
    model = Doc2Vec.load("model_test")
    
    inferred_vector_dm = model.infer_vector(doc_words=test)
    sims = model.docvecs.most_similar([inferred_vector_dm], topn=10)
    return sims

file_name = "data.csv"

raw_data = pd.read_csv(file_name)
df = pd.DataFrame(raw_data)
x_train = []

for i,row in df.iterrows():
    document = TaggededDocument(row['Comment'], tags=[i])
    x_train.append(document)
    
model = Doc2Vec(x_train,min_count=1, window = 3, size = 150, sample=1e-3, workers=4,hs=1,iter=6)
model.train(x_train, total_examples=model.corpus_count, epochs=30)
model.save('model_test')

        
#    test = ...
#    sims = test()
#    for count, sim in sims:
#        sentence = str(x_train[count])
        # sentence = x_train[count]
        # print('sentence:'+sentence)
        # print('sim:'+str(sim))
#        print(sentence, sim, len(sentence))

/opt/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:566: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/opt/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [17]:
vecs = [np.array(model.docvecs[z.tags[0]]).reshape((1, 150)) for z in x_train]  # tags[0]是根据标签找向量，reshape的意思是重新按照这个矩阵大小排列。例如 a = np.arange(6).reshape((3, 2)) 输出([[0, 1],[2, 3],[4, 5]])
X = np.concatenate(vecs)


In [14]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [18]:
X.shape

(2125056, 150)

In [21]:
kmeans = KMeans(n_clusters=10, init='k-means++', n_init=5).fit(X)


In [22]:
clist_10 = []
for label in kmeans.labels_:
    clist_10.append(label)

In [26]:
kmeans = KMeans(n_clusters=15, init='k-means++', n_init=5).fit(X)

clist_15 = []
for label in kmeans.labels_:
    clist_15.append(label)

In [28]:
kmeans = KMeans(n_clusters=20, init='k-means++', n_init=5).fit(X)

clist_20= []
for label in kmeans.labels_:
    clist_20.append(label)

In [38]:
tmp_dic = {'clus_10':clist_10,'clus_15':clist_15,'clus_20':clist_20}
clus_df = pd.DataFrame(tmp_dic, columns=['clus_10', 'clus_15', 'clus_20'])

In [39]:
docv_df = pd.DataFrame(X)
res = pd.concat([clus_df, docv_df],axis=1)


In [40]:
res

,clus_10,clus_15,clus_20,0,1,2,3,4,5,6,...,140,141,142,143,144,145,146,147,148,149
0,2,14,16,0.454749,0.075933,0.989870,0.763248,0.790644,-0.251485,-0.316166,...,0.451340,-0.205073,0.959361,0.243185,-0.960646,-1.106796,-1.120893,-0.754034,0.301842,-0.711453
1,8,8,14,0.211989,0.485131,-0.756601,0.339424,-0.324144,-2.769078,-1.416410,...,0.541693,-0.467021,-0.659951,0.601387,0.631507,0.560771,-0.648289,-1.247034,0.201610,-0.238821
2,1,13,19,0.517644,-0.483260,0.215278,1.264967,-0.322865,-1.913158,0.256509,...,-0.583523,-1.085528,1.698945,-0.438385,0.282447,0.990588,-0.360752,0.734434,0.957895,-0.641732
3,9,5,17,-0.731719,-0.488490,-0.591593,1.106247,-1.671839,0.311964,-0.216956,...,1.159797,0.346400,-2.014061,-0.528511,-1.246807,1.182403,-0.085697,1.188419,-2.269073,0.627365
4,5,5,1,0.160218,-0.414370,1.155223,-0.110876,-0.330072,0.289665,0.792875,...,0.194380,-0.244511,0.204626,-0.049117,0.685791,-0.149767,0.060965,-0.474822,-0.149615,-0.258662
5,5,13,11,0.104035,0.856233,1.013289,-0.253802,1.496202,1.689694,-0.490203,...,0.021130,-0.551046,0.404666,0.234500,-0.248470,-1.250434,0.536455,-0.117993,1.255967,-0.708034
6,5,6,17,-0.931717,0.466346,0.820053,-0.319068,-0.958943,-1.393632,0.993937,...,-0.923889,-0.264479,-1.290142,-1.570770,-0.458085,0.702444,0.403883,0.453409,-0.228465,-1.947498
7,8,11,3,0.894262,0.480287,1.392853,-0.823165,0.986922,0.650972,1.674797,...,0.688301,-0.993085,0.248002,-0.135104,0.337365,-0.442643,-1.246762,0.495055,-1.457670,-0.742216
8,6,11,11,0.484781,-0.156893,-0.570019,-0.098150,-0.078652,-0.264801,1.004178,...,-0.402525,-1.039898,-0.820029,0.548100,-1.238249,-1.091168,0.555323,-0.711731,0.009352,-0.502651
9,1,6,4,2.074618,0.186892,0.523038,-1.221658,-0.575767,-0.601467,1.831380,...,0.937187,-0.742368,-0.776615,-0.923591,1.090211,-0.154619,-1.139419,-1.251905,-1.746036,0.599412


In [41]:
name_dic = {'复仇者联盟2':'2015-05-12', '大鱼海棠':"2016-07-08", '美国队长3':"2016-05-06", '十二生肖':"2012-12-20",
            '九层妖塔':"2015-09-30", '大圣归来':"2015-07-10", '栀子花开':"2015-07-10", '夏洛特烦恼':"2015-09-30",
            '钢铁侠1':"2008-04-30", '西游降魔篇':"2013-02-10", '西游伏妖篇':"2017-01-28", '爱乐之城':"2017-02-14", 
            '泰囧':"2012-12-12", '何以笙箫默':"2015-04-30", '湄公河行动':"2016-09-30", '七月与安生':"2016-09-14",
            '复仇者联盟':"2012-05-05", '后会无期':"2014-07-24", '寻龙诀':"2015-12-18", '长城':"2016-12-16", 
            '左耳':"2015-04-24", '美人鱼':"2016-02-08", '小时代1':"2013-06-27", '小时代3':"2014-07-17", 
            '釜山行':"2016-07-20",'变形金刚4':"2014-06-27", '你的名字':"2016-12-02", '疯狂动物城':"2016-03-04"}
df['time_interval'] = df.apply(lambda x: (pd.to_datetime(x['Date']) - pd.to_datetime(name_dic[x['Movie_Name_CN']])).days, axis=1)

In [42]:
#encoding=utf-8
import jieba

stopwords = [line.strip() for line in open("stopword.txt", 'r',encoding='utf-8').readlines()] #load stopwords

def func(row):
    tmp_list = jieba.cut(row['Comment'], cut_all = False)
    mark_count1 = 0 #! ~
    mark_count2 = 0 #?
    mark_count3 = 0 #...
    word_count = 0
    outstr = ""
    for word in tmp_list:
        if (word == '!' or word == '~' or word == '！' or word =="√" or word == "★" or word =="～"):
            mark_count1 += 1
        if (word == '?'or word == '？'):
            mark_count2 += 1
        if (word == '...' or word == '......' or word == '。。。'):
            mark_count3 += 1
        if word not in stopwords:
            word_count += 1
            outstr += word + " "
    return pd.Series([outstr, mark_count1, mark_count2, mark_count3, word_count])


tmp = pd.DataFrame()
tmp = df.apply(func, axis=1)
df['new_comment'] = tmp[0]
df['mark!'] = tmp[1]
df['mark?'] = tmp[2]
df['mark...'] = tmp[3]
df['word_num'] = tmp[4]


name_dummy = pd.get_dummies(df['Movie_Name_CN'])
df = pd.concat([df,name_dummy],axis=1)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.594 seconds.
Prefix dict has been built succesfully.


In [43]:
tmp_count = df.groupby(['Movie_Name_CN'])['ID'].agg(['count']).reset_index()
tmp_count.columns = ['Movie_Name_CN', 'movie_count']
df = pd.merge(df, tmp_count, how='left', on='Movie_Name_CN')

tmp_count = df.groupby(['Movie_Name_CN', 'Date'])['ID'].agg(['count']).reset_index()
tmp_count.columns = ['Movie_Name_CN', 'Date', 'comment_day_num']
df = pd.merge(df, tmp_count, how='left', on=['Movie_Name_CN', 'Date'])

df['comment_day_prop'] = df['comment_day_num'] / df['movie_count']

In [44]:
df.columns

Index(['ID', 'Movie_Name_EN', 'Movie_Name_CN', 'Crawl_Date', 'Number',
       'Username', 'Date', 'Star', 'Comment', 'Like', 'time_interval', 'mark!',
       'mark?', 'mark...', 'word_num', 'new_comment', '七月与安生', '九层妖塔', '何以笙箫默',
       '你的名字', '十二生肖', '变形金刚4', '后会无期', '复仇者联盟', '复仇者联盟2', '夏洛特烦恼', '大圣归来',
       '大鱼海棠', '寻龙诀', '小时代1', '小时代3', '左耳', '栀子花开', '泰囧', '湄公河行动', '爱乐之城',
       '疯狂动物城', '美人鱼', '美国队长3', '西游伏妖篇', '西游降魔篇', '釜山行', '钢铁侠1', '长城',
       'movie_count', 'comment_day_num', 'comment_day_prop'],
      dtype='object')

In [45]:
df.drop(['ID', 'Movie_Name_EN', 'Movie_Name_CN', 'Crawl_Date', 'Number','Username', 'Date',  
         'Comment', 'new_comment', 'movie_count'],axis=1, inplace=True)

In [46]:
df = pd.concat([df,res],axis=1)

In [47]:
df.columns

Index([         'Star',          'Like', 'time_interval',         'mark!',
               'mark?',       'mark...',      'word_num',         '七月与安生',
                '九层妖塔',         '何以笙箫默',
       ...
                   140,             141,             142,             143,
                   144,             145,             146,             147,
                   148,             149],
      dtype='object', length=190)

In [48]:
df['Star'] = df['Star'] - 1

In [54]:
import lightgbm as lgb
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

feature = list(df.columns)
feature.remove('Star')

train = df.sample(frac=0.8, random_state=12345678)
test = df.drop(train.index)

X_train,X_test,y_train,y_test = train_test_split(train[feature],train['Star'],test_size=0.2)




In [59]:
from sklearn.model_selection import GridSearchCV

params = {'boosting_type': 'gbdt',
          'max_depth' : -1,
          'objective': 'multiclass',
          'num_class': 5,
          'num_leaves': 64,
          'learning_rate': 0.05,
          'colsample_bytree': 0.8,
          'reg_alpha': 5,
          'reg_lambda': 10,
          'min_split_gain': 0.5,
          'min_child_weight': 1,
          'min_child_samples': 5 }

# Create parameters to search
gridParams = {
    'learning_rate': [0.5, 0.005],
    'n_estimators': [40, 200, 1000],
    'num_leaves': [5, 10, 15],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'random_state' : [501], 
    'colsample_bytree' : [0.66],
    'subsample' : [0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.4],
    }

# Create classifier to use
clf = lgb.LGBMClassifier(boosting_type= 'gbdt',
          max_depth = params['max_depth'],
          min_split_gain = params['min_split_gain'],
          min_child_weight = params['min_child_weight'],
          min_child_samples = params['min_child_samples'])

grid = GridSearchCV(clf, gridParams,
                    verbose=3,
                    cv=4,
                    n_jobs=2)
# Run the grid
grid.fit(X_train, y_train)


Fitting 4 folds for each of 72 candidates, totalling 288 fits
[CV] boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=40, num_leaves=5, objective=binary, random_state=501, reg_alpha=1, reg_lambda=1, subsample=0.75 
[CV] boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=40, num_leaves=5, objective=binary, random_state=501, reg_alpha=1, reg_lambda=1, subsample=0.75 
[CV]  boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=40, num_leaves=5, objective=binary, random_state=501, reg_alpha=1, reg_lambda=1, subsample=0.75, score=0.4448709012355483, total=  47.6s
[CV] boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=40, num_leaves=5, objective=binary, random_state=501, reg_alpha=1, reg_lambda=1, subsample=0.75 
[CV]  boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=40, num_leaves=5, objective=binary, random_state=501, reg_alpha=1, reg_lambda=1, subsample=0.75, score=0.444

[CV] boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=40, num_leaves=10, objective=binary, random_state=501, reg_alpha=1, reg_lambda=1.4, subsample=0.75 
[CV]  boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=40, num_leaves=10, objective=binary, random_state=501, reg_alpha=1, reg_lambda=1.4, subsample=0.75, score=0.4547660362930502, total= 1.5min
[CV] boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=40, num_leaves=10, objective=binary, random_state=501, reg_alpha=1, reg_lambda=1.4, subsample=0.75 
[CV]  boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=40, num_leaves=10, objective=binary, random_state=501, reg_alpha=1, reg_lambda=1.4, subsample=0.75, score=0.45432647767278117, total= 1.5min
[CV] boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=40, num_leaves=10, objective=binary, random_state=501, reg_alpha=1, reg_lambda=1.4, subsample=0.75 
[CV]  boosting_ty

[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed: 18.3min


[CV]  boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=40, num_leaves=10, objective=binary, random_state=501, reg_alpha=1.2, reg_lambda=1.4, subsample=0.75, score=0.45416177711897554, total= 1.4min
[CV] boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=40, num_leaves=10, objective=binary, random_state=501, reg_alpha=1.2, reg_lambda=1.4, subsample=0.75 
[CV]  boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=40, num_leaves=10, objective=binary, random_state=501, reg_alpha=1.2, reg_lambda=1.4, subsample=0.75, score=0.45453369018558276, total= 1.5min
[CV] boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=40, num_leaves=10, objective=binary, random_state=501, reg_alpha=1.2, reg_lambda=1.4, subsample=0.75 
[CV]  boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=40, num_leaves=10, objective=binary, random_state=501, reg_alpha=1.2, reg_lambda=1.4, subsample=0.75, score

[CV] boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=200, num_leaves=5, objective=binary, random_state=501, reg_alpha=1, reg_lambda=1, subsample=0.75 
[CV]  boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=200, num_leaves=5, objective=binary, random_state=501, reg_alpha=1, reg_lambda=1, subsample=0.75, score=0.46653333725478663, total= 4.5min
[CV] boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=200, num_leaves=5, objective=binary, random_state=501, reg_alpha=1, reg_lambda=1, subsample=0.75 
[CV]  boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=200, num_leaves=5, objective=binary, random_state=501, reg_alpha=1, reg_lambda=1, subsample=0.75, score=0.46596550669396014, total= 4.4min
[CV] boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=200, num_leaves=5, objective=binary, random_state=501, reg_alpha=1, reg_lambda=1.4, subsample=0.75 
[CV]  boosting_type=gbdt

[CV]  boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=200, num_leaves=10, objective=binary, random_state=501, reg_alpha=1, reg_lambda=1.4, subsample=0.75, score=0.47167436251874945, total= 7.5min
[CV] boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=200, num_leaves=10, objective=binary, random_state=501, reg_alpha=1, reg_lambda=1.4, subsample=0.75 
[CV]  boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=200, num_leaves=10, objective=binary, random_state=501, reg_alpha=1, reg_lambda=1.4, subsample=0.75, score=0.4718138155979142, total= 5.1min
[CV] boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=200, num_leaves=10, objective=binary, random_state=501, reg_alpha=1.2, reg_lambda=1, subsample=0.75 
[CV]  boosting_type=gbdt, colsample_bytree=0.66, learning_rate=0.5, n_estimators=200, num_leaves=10, objective=binary, random_state=501, reg_alpha=1, reg_lambda=1.4, subsample=0.75, score=0.470

KeyboardInterrupt: 

In [56]:

gbm = lgb.LGBMClassifier(**grid.best_params_)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds=5)

from sklearn.metrics import roc_auc_score,accuracy_score
y_pred=gbm.predict(test[feature])
y_pred=[list(x).index(max(x)) for x in y_pred]
print(y_pred)
print(accuracy_score(test['Star'],y_pred))

NameError: name 'grid' is not defined

In [53]:
print(pd.DataFrame({
        'column': feature,
        'importance': clf.feature_importance(),
    }).sort_values(by='importance'))

              column  importance
3              mark?           0
4            mark...           0
83                44           0
53                14           0
122               83           0
126               87           0
179              140           1
99                60           1
103               64           1
155              116           1
78                39           1
95                56           1
88                49           2
89                50           2
151              112           2
167              128           2
60                21           2
57                18           2
125               86           3
124               85           3
150              111           3
163              124           3
58                19           3
96                57           3
144              105           3
182              143           3
113               74           4
97                58           4
104               65           4
170       

In [ ]:
train_data=lgb.Dataset(X_train,label=y_train)
validation_data=lgb.Dataset(X_test,label=y_test)
params={
    'learning_rate':0.1,
    'lambda_l1':0.2,
    'lambda_l2':0.2,
    'max_depth':4,
    'objective':'multiclass',
    'num_class':5,  #lightgbm.basic.LightGBMError: b‘Number of classes should be specified and greater than 1 for multiclass training‘
}
clf=lgb.train(params,train_data,valid_sets=[validation_data])
from sklearn.metrics import roc_auc_score,accuracy_score
y_pred=clf.predict(test[feature])
y_pred=[list(x).index(max(x)) for x in y_pred]
print(y_pred)
print(accuracy_score(test['Star'],y_pred))

In [75]:
min(df['comment_day_prop'])

7.2721455010871859e-06

In [68]:
tff['count'].mean()

2031.6022944550668

In [76]:
raw_data.columns


Index(['ID', 'Movie_Name_EN', 'Movie_Name_CN', 'Crawl_Date', 'Number',
       'Username', 'Date', 'Star', 'Comment', 'Like', 'time_interval', 'mark!',
       'mark?', 'mark...', 'word_num', 'new_comment'],
      dtype='object')